# Корреляционный анализ
### Вопрос 1
Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.
Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [1]:
import pandas as pd

In [8]:
df = pd.read_table('water.txt')

In [9]:
df.corr(method = 'pearson')

,mortality,hardness
mortality,1.000000,-0.654849
hardness,-0.654849,1.000000


### Вопрос 2
В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [10]:
df.corr(method = 'spearman')

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


### Вопрос 3
Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [12]:
df[df.location == 'South'].corr(method = 'pearson')

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [13]:
df[df.location == 'North'].corr(method = 'pearson')

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


### Вопрос 4
Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

$$MCC = \frac{ad - bc}{\sqrt{(a+b)(a+c)(b+d)(d+c)}}$$

In [20]:
a = 718 # woman + rare bar
b = 203 # woman + often bar
c = 515 # man + rare bar
d = 239 # man + often bar

In [26]:
mcc = (a*d - b*c)/((a+b)*(a+c)*(b+d)*(c+d))**0.5

In [27]:
round(mcc, 4)

0.109

### Вопрос 5
В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [53]:
import scipy.stats
import numpy as np

In [39]:
scipy.stats.chi2_contingency(df.loc[['woman', 'man']][['rare bar', 'often bar']])

(19.407530788543038,
 1.0558987006638725e-05,
 1,
 array([[ 677.96597015,  243.03402985],
        [ 555.03402985,  198.96597015]]))

In [34]:
df = pd.DataFrame.from_dict({'woman': {'rare bar': 718, 'often bar': 203}, 'man': {'rare bar': 515, 'often bar': 239}}).T

In [38]:
df.loc[['woman', 'man']][['rare bar', 'often bar']]

,rare bar,often bar
woman,718,203
man,515,239


### Вопрос 6
В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [41]:
n_w = (203+718)
n_m = (239+515)
p_w = 203./(203+718)
p_m = 239./(239+515)

In [43]:
delta = (p_w*(1-p_w)/n_w + p_m*(1-p_m)/n_m)**0.5

In [45]:
z = scipy.stats.norm.ppf(1 - 0.05/ 2.)

In [47]:
lower_bound = p_m - p_w - z*delta
upper_bound = p_m - p_w + z*delta
(lower_bound, upper_bound)

(0.053905233215813156, 0.13922183141523897)

### Вопрос 7
Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [50]:
p = (p_w*n_w + p_m*n_m)/(n_w + n_m)
Z = (p_m - p_w)/(p*(1-p)*(1./n_w + 1./n_m))**0.5
Z

4.46111444482329

In [51]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [54]:
proportions_diff_z_test(Z, 'two-sided')

8.1534530895766011e-06

### Вопрос 8
Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

Не доволен	Более или менее	Доволен
Не очень счастлив	197	111	33
Достаточно счастлив	382	685	331
Очень счастлив	110	342	333

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [55]:
data = {
    'unhappy': {
        'unhappy_fin': 197,
        'happy_fin': 111,
        'very happy_fin': 33
    },
    'happy': {
        'unhappy_fin': 382,
        'happy_fin': 685,
        'very happy_fin': 331
    },
    'very happy': {
        'unhappy_fin': 110,
        'happy_fin': 342,
        'very happy_fin': 333
    }
}

In [58]:
fin_df = pd.DataFrame.from_dict(data).T

In [65]:
fin_df

,happy_fin,unhappy_fin,very happy_fin
happy,685,382,331
unhappy,111,197,33
very happy,342,110,333


In [62]:
chi2 = scipy.stats.chi2_contingency(fin_df)
chi2

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 630.318542  ,  381.6251981 ,  386.0562599 ],
        [ 153.74722662,   93.08597464,   94.16679873],
        [ 353.93423138,  214.28882726,  216.77694136]]))

### Вопрос 9
На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

### Вопрос 10
Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

In [72]:
def cramers_stat(confusion_matrix):
    chi2 = scipy.stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))

In [74]:
cramers_stat(fin_df.values)

0.2412013934500338